In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np

Path.cwd()

## Analyze data from September 6 and 7

In [3]:
sep6 = '/Users/huynhphuong/Desktop/CAMCOS-after-semester/Data/Sep-06.csv'
sep6 = pd.read_csv(sep6, sep='\t', dtype ={"curblocknumber":int,'gas':float, 
                                           'gasprice':float,'type':float,'maxfeepergas':float,
                                           'basefeepergas':float,'gasused':float})

sep7 = '/Users/huynhphuong/Desktop/CAMCOS-after-semester/Data/Sep-07.csv'
sep7 = pd.read_csv(sep7, sep='\t', dtype ={"curblocknumber":int,'gas':float, 
                                           'gasprice':float,'type':float,'maxfeepergas':float,
                                           'basefeepergas':float,'gasused':float})
table = pd.concat([sep6, sep7],ignore_index = True)
table

/var/folders/h_/l0mmmvfx0d76bdhlc278_vs40000gn/T/ipykernel_738/321135221.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  sep6 = pd.read_csv(sep6, sep='\t', dtype ={"curblocknumber":int,'gas':float,
/var/folders/h_/l0mmmvfx0d76bdhlc278_vs40000gn/T/ipykernel_738/321135221.py:7: DtypeWarning: Columns (4,8,9,10,19) have mixed types. Specify dtype option on import or set low_memory=False.
  sep7 = pd.read_csv(sep7, sep='\t', dtype ={"curblocknumber":int,'gas':float,


,detecttime,hash,status,region,reorg,replace,curblocknumber,failurereason,blockspending,timepending,...,network,type,maxpriorityfeepergas,maxfeepergas,basefeepergas,dropreason,rejectionreason,stuck,gasused,detect_date
0,2023-09-06T00:00:00.000Z,0x01b5bc3adca94b8d230ba4fa4ee825f5664211eac71e...,evicted,us-east-1,NaN,NaN,18073700,none,NaN,NaN,...,main,NaN,NaN,NaN,2.749919e+10,underpriced-txs,NaN,NaN,NaN,2023-09-06
1,2023-09-06T00:00:00.000Z,0x3a109b8ae8001744bb9e48e92cdab33a325802efc043...,pending,us-east-1,NaN,NaN,18073700,none,NaN,NaN,...,main,2.0,100000000.0,6.041284e+10,2.749919e+10,NaN,NaN,False,-1.0,2023-09-06
2,2023-09-06T00:00:00.001Z,0xa1640823cc1c5ae53b741bf267acd1f9b82bee442c32...,evicted,ap-southeast-1,NaN,NaN,18073700,none,NaN,NaN,...,main,0.0,NaN,NaN,2.749919e+10,future transaction tries to replace pending,NaN,False,NaN,2023-09-06
3,2023-09-06T00:00:00.006Z,0xc3a65e98896bca2658f5f5c14ec79af2d7b2c9b6b6e4...,pending,us-east-1,NaN,NaN,18073700,none,NaN,NaN,...,main,2.0,94000000.0,3.309302e+10,2.749919e+10,NaN,NaN,False,-1.0,2023-09-06
4,2023-09-06T00:00:00.013Z,0x0a2edc64439d774bd3d6f422f0bc5f34089ffe0c3a55...,evicted,us-east-1,NaN,NaN,18073700,none,NaN,NaN,...,main,NaN,NaN,NaN,2.749919e+10,underpriced-txs,NaN,NaN,NaN,2023-09-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21238905,2023-09-07T23:59:59.986Z,0x50963ff466addf38b81d1bae2e4b0f1261e2332f27af...,confirmed,ap-southeast-1,NaN,NaN,18087986,none,1.0,6671.0,...,main,0.0,NaN,NaN,1.287302e+10,NaN,NaN,False,115927.0,2023-09-07
21238906,2023-09-07T23:59:59.986Z,0x9c7ffc99f0b453c0f08ac59da48f509bb1518c2bf142...,confirmed,ap-southeast-1,NaN,NaN,18087986,none,1.0,9849.0,...,main,2.0,27192919646.0,2.719292e+10,1.287302e+10,NaN,NaN,False,48885.0,2023-09-07
21238907,2023-09-07T23:59:59.997Z,0xd8ddbb8ba2fa6d52423741d887b083ea1a1ec10739ea...,pending,us-east-1,NaN,NaN,18087985,none,NaN,NaN,...,main,2.0,5000000000.0,2.029184e+10,1.284868e+10,NaN,NaN,False,-1.0,2023-09-07
21238908,2023-09-07T23:59:59.998Z,0x21b92db7851d08541454d4a58c362499e17139658a86...,pending,eu-central-1,NaN,NaN,18087985,none,NaN,NaN,...,main,2.0,3000000000.0,2.230954e+10,1.284868e+10,NaN,NaN,False,-1.0,2023-09-07


In [4]:
# table = df.copy()
table.columns

Index(['detecttime', 'hash', 'status', 'region', 'reorg', 'replace',
       'curblocknumber', 'failurereason', 'blockspending', 'timepending',
       'nonce', 'gas', 'gasprice', 'value', 'toaddress', 'fromaddress',
       'input', 'network', 'type', 'maxpriorityfeepergas', 'maxfeepergas',
       'basefeepergas', 'dropreason', 'rejectionreason', 'stuck', 'gasused',
       'detect_date'],
      dtype='object')

# 1. Column information 
also found at https://docs.blocknative.com/mempool-data-program#data-schema

#### 1. Transaction status: 
- rejected - denied entry into the mempool 
- evicted -  exitting the mempool 
- confirmed - finalized on chain 
- pending - pending in the mempool
- speedup - replacing existing mempool tx
- cancel - replacing existing mempool tx
- failed - finalized on chain 

#### 2. For each status, there are corresponding columns
- rejected --> rejectionreason 
- evicted --> dropreason 
- confirmed --> gasused 
- pending --> gasused = -1 
- speedup --> replace 
- cancel --> replace 
- failed --> failurereason 
 

#### 3. Dypes

* **detecttime** ( datetime) - timestamp that txn was detected in the mempool 
* **hash** (string) -  unique indentifier hash for a txn
* **status** (string) - status of a txn
* **region** (string) - region of the node that detected the txn (us-east-1, eu-central-1, ap-southeast-1)
* **reorg** (string) - refer to block hash of the reorg, if there was one
* **replace** (string) - if a txn was replaced (speedup/cancel), show the txn hash of the replacement
* **curblocknumber** (int)- the block number the event was detected in 
* **failurereason** (string) - reason of a failed txn
* **blockspending** (int) - number of blocks that txn a txn was waiting to get on chain after finalized (confirmed/failed)
* **time spending** (big int) - time in milisecinds that txn was waiting to get on chain
* **nonce** (int) - unique number count the number of txns sent from a given address
* **gas** (int) - a txn fee to incentivize miners (denom is gwei or 1e9ETH)(also the gas limit for a txn set by the users)
* **gasprice** (int) - price offered to miner to purchase gas (denom is gwei or 1e9ETH) (only for txns type 0 and 1)
* **value** (int) - amount of ETH (denom is gwei) transferred
* **toadress** (string)
* **fromadress** (string)
* **input** (string) - additional data that can be attached to a transaction. This field can be used to tell a smart contract to execute a function
* **network** (string) - ETH network used
* **type** (int) - type 0,1,2
* **maxpriorityfeepergas** (int) - max possible tip (actual tip = maxfee - basefee) (denom is gwei)
* **maxfeepergas** (int) - max possible gas a sender is willing to pay including basefee and tip (why must be lower than the maxpriorityfeepergas?)
* **basefeepergas** (int) - burned fee (denom is gwei)
* **dropreason** (string)
* **rejectionreason** (string)
* **stuck** - no info, dropped
* **gasused** (int) - actuan gas consumed (denom is gwei)
* d**etect_date** (string) 

#### 4. Current dtypes:
    detecttime               object 
    hash                     object
    status                   object
    region                   object
    reorg                    string
    replace                  object
    curblocknumber            int64
    failurereason            object
    blockspending            object
    timepending              object
    nonce                    object
    gas                     float64
    gasprice                float64
    value                    object
    toaddress                object
    fromaddress              object
    input                    object
    network                  object
    type                    float64
    maxpriorityfeepergas     object
    maxfeepergas            float64
    basefeepergas           float64
    dropreason               object
    rejectionreason          object
    stuck                    object
    gasused                 float64
    detect_date              object
    
#### 5. Dtypes needed to be fixed (maybe data entry error)
    - region: drop rows that are not the three region 
    - blockspending
    - timepending
    - nonce
    - maxpriorityfeepergas
    
#### 6. Drop columns
    - stuck: no information
    - network: only one value 'main, 
    - gasprice: if analyzing post EIP txns (type 2 txns)

#### 7. Some Observations
    - There are some duplicate hash for the same txns

# 2. Processing Data
### 2.1 drop columns and data entry error rows

In [5]:
table = table.drop(columns = ["stuck","network",'gasprice'])
table = table[table.region != '7']
# table = table.reset_index(drop=True)
table.head(5)

,detecttime,hash,status,region,reorg,replace,curblocknumber,failurereason,blockspending,timepending,...,fromaddress,input,type,maxpriorityfeepergas,maxfeepergas,basefeepergas,dropreason,rejectionreason,gasused,detect_date
0,2023-09-06T00:00:00.000Z,0x01b5bc3adca94b8d230ba4fa4ee825f5664211eac71e...,evicted,us-east-1,NaN,NaN,18073700,none,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.749919e+10,underpriced-txs,NaN,NaN,2023-09-06
1,2023-09-06T00:00:00.000Z,0x3a109b8ae8001744bb9e48e92cdab33a325802efc043...,pending,us-east-1,NaN,NaN,18073700,none,NaN,NaN,...,0xca55528c7daa0a21ca60c74d5e725b8110133672,0x095ea7b3000000000000000000000000b51785051099...,2.0,100000000.0,6.041284e+10,2.749919e+10,NaN,NaN,-1.0,2023-09-06
2,2023-09-06T00:00:00.001Z,0xa1640823cc1c5ae53b741bf267acd1f9b82bee442c32...,evicted,ap-southeast-1,NaN,NaN,18073700,none,NaN,NaN,...,0xbac5cd6dcd456ebdaf03b0448388f111d9ef1a3e,0xa9059cbb000000000000000000000000c8989e28a908...,0.0,NaN,NaN,2.749919e+10,future transaction tries to replace pending,NaN,NaN,2023-09-06
3,2023-09-06T00:00:00.006Z,0xc3a65e98896bca2658f5f5c14ec79af2d7b2c9b6b6e4...,pending,us-east-1,NaN,NaN,18073700,none,NaN,NaN,...,0x0278e6a96f850a74daef69ccd230e89575111d60,0x,2.0,94000000.0,3.309302e+10,2.749919e+10,NaN,NaN,-1.0,2023-09-06
4,2023-09-06T00:00:00.013Z,0x0a2edc64439d774bd3d6f422f0bc5f34089ffe0c3a55...,evicted,us-east-1,NaN,NaN,18073700,none,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.749919e+10,underpriced-txs,NaN,NaN,2023-09-06


### 2.2 Fix dtypes

In [6]:
table.maxpriorityfeepergas = table.maxpriorityfeepergas.astype(float)
table.blockspending = table.blockspending.astype(float)
table.timepending = table.timepending.astype(float)
table.nonce = table.nonce.astype(float)
table.dtypes

detecttime               object
hash                     object
status                   object
region                   object
reorg                    object
replace                  object
curblocknumber            int64
failurereason            object
blockspending           float64
timepending             float64
nonce                   float64
gas                     float64
value                    object
toaddress                object
fromaddress              object
input                    object
type                    float64
maxpriorityfeepergas    float64
maxfeepergas            float64
basefeepergas           float64
dropreason               object
rejectionreason          object
gasused                 float64
detect_date              object
dtype: object

# 3. Filtering data
### 3.1 
- Only US-east-1 region
- Only confirmed/failed txns
- Only type 2 txns
- Remove duplicate (some confirmed txns have dumplicated rows)

In [10]:
table = table[(table.status == "confirmed") | (table.status == "failed")]
table = table[(table["region"] == "us-east-1") & (table["type"] == 2)]
print("Number of duplicate: ", sum(table.hash.duplicated())) # check for reason of duplication
table = table.drop_duplicates(subset=['hash'], keep='first')
table = table.reset_index(drop = True)
table.head(10)

Number of duplicate:  38601


,detecttime,hash,status,region,reorg,replace,curblocknumber,failurereason,blockspending,timepending,...,fromaddress,input,type,maxpriorityfeepergas,maxfeepergas,basefeepergas,dropreason,rejectionreason,gasused,detect_date
0,2023-09-06T00:00:01.791Z,0xc531d2d5c0c1bbaa2b87f350b55b380c230c86e4fb01...,confirmed,us-east-1,NaN,NaN,18073701,none,0.0,0.0,...,0x0ef774ee34550a6d3963e31d515b241d34334c68,0x095ea7b3000000000000000000000000000000000022...,2.0,1.000000e+08,3.842216e+10,2.722687e+10,NaN,NaN,46280.0,2023-09-06
1,2023-09-06T00:00:01.791Z,0x36c5954ccaacd36884b36b5d3c9626d0991c8d8df292...,failed,us-east-1,NaN,NaN,18073701,"Reverted: \0x849eaf98\""""",1.0,6168.0,...,0x42b163949559faa2d9b29cfcaca16fb4eca3e13d,0x3593564c000000000000000000000000000000000000...,2.0,1.000000e+08,3.898204e+10,2.722687e+10,NaN,NaN,188260.0,2023-09-06
2,2023-09-06T00:00:01.791Z,0xe31a1b2b66795faaa0cd073cd3b7b645afbb99ef0364...,confirmed,us-east-1,NaN,NaN,18073701,none,2.0,12958.0,...,0x1c163b72d5a6bca2fc0535a1df2bbc39fbaed2f5,0x,2.0,1.000000e+08,3.986823e+10,2.722687e+10,NaN,NaN,21000.0,2023-09-06
3,2023-09-06T00:00:01.791Z,0xa164b33c1ff41b2048efb37ca08d8ec45aa36d6bc64d...,confirmed,us-east-1,NaN,NaN,18073701,none,0.0,0.0,...,0x0055f133ff5aed42c0074b2f1844a35d7ed3f860,0x8ee938a9000000000000000000000000000000000000...,2.0,6.000000e+09,4.269425e+10,2.722687e+10,NaN,NaN,153759.0,2023-09-06
4,2023-09-06T00:00:01.791Z,0x4408dcd2f9ff377ba6bee4f06dd6da265a3bf9c01634...,confirmed,us-east-1,NaN,NaN,18073701,none,1.0,8975.0,...,0xe67069d42b3802fd45429c42131de35cc9b2bf23,0x095ea7b30000000000000000000000005125109f0436...,2.0,1.000000e+08,3.842216e+10,2.722687e+10,NaN,NaN,46195.0,2023-09-06
5,2023-09-06T00:00:01.791Z,0xa75e64cb8099501e3845fb664ed8d87e3fccd4cba47d...,confirmed,us-east-1,NaN,NaN,18073701,none,1.0,11588.0,...,0x83a69d9e4988d7bfafa9325294d172ff16a39534,0x791ac947000000000000000000000000000000000000...,2.0,3.000000e+09,4.384030e+10,2.722687e+10,NaN,NaN,137638.0,2023-09-06
6,2023-09-06T00:00:01.791Z,0x378251110455e11ba352f404135fb6a0b42c6a9624cd...,confirmed,us-east-1,NaN,NaN,18073701,none,1.0,3187.0,...,0xe570dd2e4c3932dd8a847ffe5299cf6812b20dd6,0x095ea7b3000000000000000000000000000000000022...,2.0,1.000000e+08,3.842216e+10,2.722687e+10,NaN,NaN,46540.0,2023-09-06
7,2023-09-06T00:00:01.791Z,0xa8221d27437dfa74cd0a503f8fa86a4769e9c6160f33...,confirmed,us-east-1,NaN,NaN,18073701,none,1.0,8963.0,...,0x850887e9f70544eefb6931ef3ec3aaf881b0a7bd,0xa9059cbb000000000000000000000000f89d7b9c864f...,2.0,2.000000e+09,2.500000e+11,2.722687e+10,NaN,NaN,29688.0,2023-09-06
8,2023-09-06T00:00:01.791Z,0x8644b246e402d170a1bd4184ec09993a7e6be99c2345...,confirmed,us-east-1,NaN,NaN,18073701,none,2.0,14122.0,...,0xcca788316a3025c44891f3eb60cea3b317ea1b5f,0x0000018ba24ef789000000000000000000000000cca7...,2.0,1.000000e+08,3.135259e+10,2.722687e+10,NaN,NaN,144983.0,2023-09-06
9,2023-09-06T00:00:01.791Z,0x80b161e1e8457373bb12150578af413daa69b8bfbb6d...,confirmed,us-east-1,NaN,NaN,18073701,none,2.0,13832.0,...,0xcfc0f98f30742b6d880f90155d4ebb885e55ab33,0xa9059cbb000000000000000000000000af1640807720...,2.0,3.002000e+09,3.310380e+10,2.722687e+10,NaN,NaN,63209.0,2023-09-06


### 3.2 Exporting the csv file

In [12]:
table.to_csv("confirmed-failed-txn.csv")